In [ ]:
from pymongo import MongoClient
import pprint

In [ ]:
address= "mongodb://localhost:27017/"

In [ ]:
client = MongoClient(address)

In [ ]:
dbs = client.list_database_names()

In [ ]:
dbs

In [ ]:
test_db = client.TohidDb

In [ ]:
collections = test_db.list_collection_names()

In [ ]:
print(collections)

# Insert Document

In [ ]:
def insert_test_doc():
    collection = test_db.tutorial2
    test_document = {
        "name" : "Tohid",
        "type": "Test"
    }
    inserted_id = collection.insert_one(test_document).inserted_id
    print(inserted_id)

In [ ]:
insert_test_doc()

In [ ]:
production = client.production
person_collection = production.person_collection

In [ ]:
def create_documents():
    first_names = ["Tim","Sarah","Jennifer","Jose","Brad","Allen"]
    last_names = ["Rusica","Smith","Bart","Cater","Pit","Geral"]
    ages = [21,40,23,19,34,67]
    
    docs = []
    for first_name,last_name,age in zip(first_names,last_names,ages):
        doc = {"first_name": first_name, "last_name": last_name, "age": age}
        docs.append(doc)
        #person_collection.insert_one(doc)
    person_collection.insert_many(docs)

In [ ]:
create_documents()

# Reading Documents

In [ ]:
printer = pprint.PrettyPrinter()

In [ ]:
def find_all_people():
    people = person_collection.find()
    
    for person in people:
        printer.pprint(person)

In [ ]:
find_all_people()

In [ ]:
tim = person_collection.find_one({"first_name":"Tim"})
printer.pprint(tim)

In [ ]:
tim = person_collection.find({"first_name":"Tim"})
for p in tim:
    printer.pprint(p)

In [ ]:
def count_all_people():
    count = person_collection.count_documents(filter={})
    print("Number of People",count)
count_all_people()

In [ ]:
def get_person_by_id(person_id):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    
    person = person_collection.find_one({"_id":_id})
    
    printer.pprint(person)
get_person_by_id("63f45b71836c4fa1b08a98f0")

In [ ]:
def get_age_range(min_age,max_age):
    query = {"$and":[{"age":{"$gte":min_age}},{"age":{"$lte":max_age}}]}
    people = person_collection.find(query).sort("age")
    for p in people:
        printer.pprint(p)
get_age_range(20,35)

In [ ]:
def project_columns():
    columns = {"_id":0 , "first_name":1 , "last_name":1}
    people = person_collection.find({},columns)
    for p in people:
        printer.pprint(p)
project_columns()

# Updating Documents

In [ ]:
def update_person_by_id(person_id):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    
#     all_updates = {
#         "$set": {"new_field": True}, 
#         "$inc": {"age": 1},
#         "$rename": {"first_name": "first","last_name":"last"}
#     }
#     person_collection.update_one({"_id": _id}, all_updates)

    person_collection.update_one({"_id": _id},{"$unset":{"new_field":""}})
update_person_by_id("63f45b71836c4fa1b08a98ee")

In [ ]:
def replace_one(person_id):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    
    new_doc = {
        "first_name": "new first name",
        "last_name": "new last name",
        "age": 100
    }
    
    person_collection.replace_one({"_id": _id},new_doc)
replace_one("63f45b71836c4fa1b08a98ef")

# Deleting Documents

In [ ]:
def delete_doc_by_id(person_id):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    
    person_collection.delete_one({"_id": _id})
delete_doc_by_id("63f45b71836c4fa1b08a98ef")

# Relationship

In [ ]:
address = {
    "_id":"62475964011a9126a4cebeb7",
    "street" : "Bay Street",
    "number" : 2706,
    "city": "San Francisco",
    "country": "United States",
    "zip": "94107",
    #"owner_id": "62475964011a9126a4"
}

# person = {
#     "_id": "62475964011a9126a4",
#     "first_name": "John"
# }

In [ ]:
def add_address_embed(person_id,address):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    
    person_collection.update_one({"_id": _id},{"$addToSet":{"addresses":address}})
    
add_address_embed("63f45b71836c4fa1b08a98ee",address)   

In [ ]:
def add_address_relationship(person_id,address):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    address = address.copy()
    address["owner_id"] = person_id
    
    address_collection = production.address
    address_collection.insert_one(address)
    
add_address_relationship("63f45b71836c4fa1b08a98f0",address)  